In [1]:
import tensorflow.keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout

from tensorflow.keras.utils import to_categorical

import numpy as np
import nltk
import matplotlib.pyplot as plt

In [2]:
#load and clean data
raw = open("data.txt", "r")

lines = []
for line in raw:
    if str(line) != '\n':
        lines.append(str(line))

#putting the entirety of the input data into one string
text = ""
for line in lines:
    text += line
tokenized = nltk.word_tokenize(text)

#converting the text into numbers to be processed by the embedding layer of the model
words = [] #one of each word in tokenized will be in here
filtering = lambda x : not x in words #for finding if the word should be added to the words array
find = lambda x : words.index(x) if x in words else -1 #convert each word into a number. -1 means that the item isn't in the vocabulary

for word in tokenized:
    if filtering(word):
        words.append(word)

#x data is every single word in the data set, in order
#y data is every single word that comes after the corresponding x value
x = []
y = []
for i in range(len(tokenized)):
    x.append(find(tokenized[i]))
    if i+1 < len(tokenized):
        y.append(find(tokenized[i+1]))
    else:
        y.append(find("."))

x = np.asarray(x)
y = np.asarray(y)
y = to_categorical(y)

print(x.shape)
print(y.shape)

vocab_size = len(words) + 1

(24496,)
(24496, 4198)


In [3]:
#create and compile model
model = Sequential()

model.add(Embedding(vocab_size, 64, input_length=(1))) #embedding: size of vocabulary, dimension of each value
model.add(LSTM(40))
model.add(Dropout(0.01))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.01))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 64)             268736    
                                                                 
 lstm (LSTM)                 (None, 40)                16800     
                                                                 
 dropout (Dropout)           (None, 40)                0         
                                                                 
 dense (Dense)               (None, 32)                1312      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 4198)              138534    
                                                                 
Total params: 425,382
Trainable params: 425,382
Non-trai

In [5]:
#train model
hist = model.fit(x, y, epochs=500, verbose=1, batch_size=10)
plt.title("loss")
plt.plot(hist.history['loss'])
model.save("shakespear2.0.h5")

Epoch 1/500
2450/2450 [==============================] - 16s 6ms/step - loss: 6.5204 - accuracy: 0.0794
Epoch 2/500
2450/2450 [==============================] - 16s 6ms/step - loss: 6.2421 - accuracy: 0.0843
Epoch 3/500
2450/2450 [==============================] - 16s 6ms/step - loss: 6.0080 - accuracy: 0.0958
Epoch 4/500
2450/2450 [==============================] - 16s 6ms/step - loss: 5.7780 - accuracy: 0.1053
Epoch 5/500
2450/2450 [==============================] - 16s 6ms/step - loss: 5.5437 - accuracy: 0.1258
Epoch 6/500
2450/2450 [==============================] - 16s 6ms/step - loss: 5.3441 - accuracy: 0.1413
Epoch 7/500
2450/2450 [==============================] - 16s 6ms/step - loss: 5.1744 - accuracy: 0.1535
Epoch 8/500
2450/2450 [==============================] - 16s 6ms/step - loss: 5.0267 - accuracy: 0.1657
Epoch 9/500
2450/2450 [==============================] - 16s 6ms/step - loss: 4.9001 - accuracy: 0.1735
Epoch 10/500
2450/2450 [==============================] - 16s 6m

2450/2450 [==============================] - 16s 6ms/step - loss: 3.5776 - accuracy: 0.2427
Epoch 80/500
2450/2450 [==============================] - 16s 6ms/step - loss: 3.5734 - accuracy: 0.2419
Epoch 81/500
2450/2450 [==============================] - 16s 6ms/step - loss: 3.5702 - accuracy: 0.2425
Epoch 82/500
2450/2450 [==============================] - 16s 6ms/step - loss: 3.5699 - accuracy: 0.2421
Epoch 83/500
2450/2450 [==============================] - 16s 6ms/step - loss: 3.5668 - accuracy: 0.2427
Epoch 84/500
2450/2450 [==============================] - 16s 6ms/step - loss: 3.5676 - accuracy: 0.2450
Epoch 85/500
2450/2450 [==============================] - 16s 6ms/step - loss: 3.5577 - accuracy: 0.2440
Epoch 86/500
2450/2450 [==============================] - 16s 6ms/step - loss: 3.5639 - accuracy: 0.2427
Epoch 87/500
2450/2450 [==============================] - 16s 6ms/step - loss: 3.5605 - accuracy: 0.2449
Epoch 88/500
2450/2450 [==============================] - 16s 6ms/st

KeyboardInterrupt: 

In [5]:
#test model